# Implicit representation for mesh reconstruction with Point Clouds

In this lab work we will reconstruct shapes from point sets with and without their normal information.
Each network/method will output the distance or signed distance and one can extract the surface with Marching
cubes, following these steps :
- Use the trained network to compute the values of the signed distance on a grid
- Extract the 0 levelset (marching_cubes method of the mcubes library)
- Save/visualize the mesh (export_obj method of the mcubes library)

In [ ]:
!jupyter server list

In [ ]:
!pip install potpourri3d
!pip install git+https://github.com/skoch9/meshplot.git
!pip install pythreejs
!pip install pymcubes

In [ ]:
!wget https://www.lix.polytechnique.fr/~pierson/cours/tp_sdf_material.zip

In [ ]:
!unzip -o tp_sdf_material.zip
!ls

In [ ]:
!pip install PyMCubes==0.1.4

In [ ]:

import numpy as np
import mcubes
import plot_utils as plu
from mesh_utils.mesh import TriMesh
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Traditional reconstruction approach
This method is a "historical" method (<a href="https://dl.acm.org/doi/abs/10.1145/133994.134011">link</a>) for reconstructing a surface from a set of points. It consists in taking an oriented point cloud $(x_i , n_i )$, and estimating for any arbitrary point $x$ in the ambient space a signed distance function as : $u(x) = ± min_i ∥x_i − x∥$

The sign is given by the sign of the scalar product $\langle x - x_i, n_i \rangle$.

The original method starts with unoriented point clouds and devises a clever way to estimate the normal direction and their orientation. Here, for simplicity, we start with oriented points.

In [ ]:
def get_pc(path):
    ## Load the oriented point set. You can use the function np.loadtxt
    xyz=np.loadtxt(path)
    point_cloud=xyz[:,:3]
    normals=xyz[:,3:]
    return point_cloud, normals

In [ ]:
pc, normals = get_pc("armadillo_sub.xyz")
normals.shape

In [ ]:
plu.plot_pc(pc, point_size=2,cmap=normals) #You can put cmap = normals to see normals orientation as color

The second step is to compute the sdf based on the set of points. You will first need to build a grid of points (using e.g. np.meshgrid), and then to compute the sdf to the set of points. Don't forget to adapt the limits of the grid to the size of the point cloud! For the distance, use an efficient way to compute the distance (look at solutions of previous labs to get an idea).

In [ ]:
from scipy.spatial import cKDTree
def Query_CPU(xyz_query, xyz_search, K):
    # Use scipy Kdtree to perform the query
    # XYZ MUST BE OF dimension 2 not 3
    kdtree = cKDTree(xyz_search)
    # TODO: you may need to tweak the number of num_workers, 1 is really long but -1 is can OOM
    distances, neighbors = kdtree.query(
        xyz_query, k=K, workers=-1
    )
    return distances, neighbors

def compute_nearest_neighbor(X, Y):
    """
    Compute the nearest neighbor in Y for each point in X

    Parameters:
    -----------
    X : (n, d) array of points
    Y : (m, d) array of points

    Returns:
    --------
    nearst_neighbor : (n,) array of indices of the nearest neighbor in Y for X
    """
    return Query_CPU(xyz_query=X,xyz_search=Y,K=1)[1]

    

In [ ]:

from scipy import spatial
def compute_sdf(point_cloud, normals, points_query):
    ## Compute SDF on points_query from the shape defined by point_cloud and normals
    print("point clouds",point_cloud.shape)
    print("query",points_query.shape)
    distances,neighbors=Query_CPU(xyz_query=points_query,xyz_search=point_cloud,K=1)
    print("point_clouds neighbors",point_cloud[neighbors].shape)
    # print("test",np.dot(points_query-point_cloud[neighbors],normals[neighbors]))
    print("normals",normals[neighbors].shape)
    affine_vector=points_query-point_cloud[neighbors]
    #np.tensordot(affine_vector,normals[neighbors],axes=0)
    return distances*np.sign(
       np.sum(affine_vector*normals[neighbors],axis=1))    

def compute_sdf_grid(point_cloud, normals, grid_size=40):
    ## Compute SDF on a XYZ grid. First generate the grid (it has to enclose the point cloud)
    ## Then compute the sdf
    #compute the enclosing grid
    max_x=np.max(point_cloud,axis=0)
    min_x=np.min(point_cloud,axis=0)
    liste_grid=[]
    for i in range(len(min_x)):
        liste_grid.append(np.linspace(min_x[i]+1,max_x[i]+1,grid_size))
    grid_x,grid_y,grid_z=np.meshgrid(liste_grid[0],liste_grid[1],liste_grid[2])
    # points_query=grid[0].shape
    points_query=np.array((grid_x.ravel(), grid_y.ravel(),grid_z.ravel())).T
    sdf=compute_sdf(point_cloud=point_cloud,normals=normals,points_query=points_query)

    
    return sdf # shape (grid_size, grid_size, grid_size)
sdf = compute_sdf_grid(pc, normals)

In [ ]:
grid_size=40
sdf = compute_sdf_grid(pc, normals,grid_size=grid_size)

In [ ]:
sdf.reshape(grid_size,grid_size,grid_size).shape
sdf=sdf.reshape(grid_size,grid_size,grid_size)

In [ ]:
triangles.shape

In [ ]:
vertices, triangles = mcubes.marching_cubes(sdf,0)
mesh = TriMesh(vertices, triangles)
print("mesh",mesh)
#mcubes.export_obj(vertices, triangles, 'result_hoppe.obj')
plu.plot(mesh)

You can try different grid sizes, but do not increase too much its size to avoid memory issues

# DeepSDF

This method (see <a href="https://arxiv.org/pdf/1901.05103">link</a>) consists of representing the SDF as a function (x, y, z) -> sdf, parameterized by a neural network.

We first build the network according to the following figure

![title](img/TD5_sdf.png)

The activations are ReLUs, except for the last one, defined as $\phi(a) = \text{tanh}(a)$.

Moreover, the networks have specific initialization (**except the last one**): the weights of size $n \times n$ are initialized according to the following $\mathcal{N}\left(0, \sqrt{\frac{2}{n}}\right)$ law, and the bias are initalized to 0 (except for the last linear layer). You can access to a Linear layer weight, and bias via layer..weight.data, and layer.bias.data, or use <a href="https://pytorch.org/docs/stable/nn.init.html">nn.init<a> on layer.weight, layer.bias

In [ ]:
class SDFNet(nn.Module):
    def __init__(self, ninputchannels, dropout=0.2, gamma=0, sal_init=False, eik=False):
        super(SDFNet, self).__init__()
        ## Prepare the layers
        ## Don't forget to initialize your weights correctly.

        ## gamma, sal_init, eik are for later
        self.gamma=gamma
        self.eik = eik
        


        #custom weights init

    def forward(self,x):
        ## Logic of the neural network
        ## You can add dropout if you want
        return x

### Loss function

The loss is computed by sampling random points in the ambient space (set X), computing their ground truth SDF (using part one), and computing the distance between computed and ground truth sdf:

$$
\mathcal{L}(\theta) = \mathbb{E}_{x \sim X} [|\text{clamp}(u_\theta(x), \delta) - \text{clamp}(\text{SDF}_{\text{gt}}(x), \delta)|]
$$

where $\text{clamp}(x, \delta) := \min(\delta, \max(−\delta, x))$ (you can use torch.clamp). To understand the signification of parameter $\delta$, read carefully paragraph 3 of the paper.

In [ ]:
def evaluate_loss(net, pts_gt, sdf_gt, device, lpc, batch_size=2000, delta = 0.1):
    ## For this function, you need to sample batch_size number of points
    ## From pts_gt. Evaluate the sdf at those points and compute the loss
    ## compared to sdf_gt (be careful to select the same points between pts_gt and sdf_gt)

    # Select points
    

    # compute and store the losses
    loss = 

    # append all the losses
    lpc.append(float(loss.item()))

    return loss

### Training the SDF

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_normalized_pointcloud(point_cloud, margin=0.05):
    ## Return the same point cloud, scaled such that 
    ## x,y,z values are between -1+margin and 1-margin
    #compute the enclosing grid

    #normalize the points
    return pc_normed

def compute_gt_sdf(point_cloud, normals, n_points=1000000):
    ## Sample a n_points points in with XYZ coordinates between -1 and 1
    ## Then use compute_sdf to get sdf_gt

    p_norm = get_normalized_pointcloud(point_cloud, margin=0.00001)
     #preparing gt points:

    gtp = 
    sdf_gt = compute_sdf(p_norm, normals, gtp)
    return sdf_gt, gtp

In [ ]:
n_points = 100000
sdf_gt, gtp = compute_gt_sdf(pc, normals, n_points)

In [ ]:
print(device)
print(sdf_gt.shape, gtp.shape) ## Should be same shape
print(np.isclose(gtp.max(), 1, 1e-3), np.isclose(gtp.min(), -1, 1e-3)) ## Should be equal to one

In [ ]:
def training_sdf(sdf_gt, gtp):
    geomnet = SDFNet(3)
    geomnet.to(device)
    gtpoints = torch.from_numpy(gtp).float().to(device)
    gtsdf = torch.from_numpy(sdf_gt).float().to(device)

    lpc = []

    optim = torch.optim.Adam(params = geomnet.parameters(), lr=1e-5)

    nepochs=10000
    pbar = tqdm(total=nepochs,
                desc="Training")

    for epoch in range(nepochs):
        loss = evaluate_loss(geomnet, gtpoints, gtsdf, device, lpc, delta = 0.1, batch_size=2500)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if epoch % 100 == 0:
        #     print(f"Epoch {epoch}/{nepochs} - loss : {loss.item()}")
            pbar.set_postfix({'loss': loss.item()})
        pbar.update(1)
    return lpc, geomnet


In [ ]:
loss_, net_sdf = training_sdf(sdf_gt, gtp)
## If the training is slow (hours), change you execution environment to GPU!

In [ ]:
# Check that the network learned something
plt.figure(figsize=(6,4))
plt.yscale('log')
plt.plot(loss_, label = 'Point cloud loss ({:.2f})'.format(loss_[-1]))
plt.xlabel("Epochs")
plt.legend()
plt.show()

## Reconstruct the shape
Code the function compute_deepsdf that compute sdf on a grid using a trained sdf network

In [ ]:
def compute_deepsdf(net, grid_size=40):
    net.eval()
    

    v = # point cloud definition (more than one_line, reshape it to (something, 3))
    queries = torch.from_numpy(v).float().to(device)
    with torch.no_grad():
        distance = net(queries).detach().cpu().numpy()
    u = np.reshape(distance,(grid_size,grid_size,grid_size))
    return u

In [ ]:
u = compute_deepsdf(net_sdf, 40)
vertices, triangles = mcubes.marching_cubes(u,0)
mesh = TriMesh(vertices, triangles)
mcubes.export_obj(vertices, triangles, 'result_deepsdf.obj')
plu.plot(mesh)

# Unsigned Distance Function

In this case (paper <a href="https://arxiv.org/pdf/1911.10414">link</a>), the objective is to learn directly on raw point clouds, without pre-processing to predicts normals/orientation of the shape. To reach this objective, the authors notice the following:
- Using the usigned distance function (absolute value of the predicted SDF) is then necessary
- Carefully choosing the points where to predict the distance is crucial
- Weights initialization is to be changed

The modification to SDF is simple : the loss is now computed by sampling points around each data point $x_i$ , following a centered Gaussian distribution of variance $\sigma²$:

$$
\mathcal{L}(\theta) = \sum_i \mathbb{E}_{x \sim \mathcal{N}(x_i, \sigma^2)}[(|u_{\theta}(x)| - |\text{SDF}_{\text{GT}}](x)|)^2)]
$$

where $\sigma$ is a parameter that you can play with, and $\text{SDF}_{\text{GT}}(x)$ is simply $\text{dist}(x_i, x)$.

The last linear layer is now initialized too, with weights following $\mathcal{N}\left(0, 2\sqrt{\pi}\right)$ law and bias initialized to -1. The last layer activation is now $\phi(a) = \text{tanh}(a) + \gamma a$. Gamma parameter now equals to 0.5.

## Neural network
- Modify SDFNet with sal_init, such that when sal_init=True, the last layer is initialized properly
- Take in account gamma parameter in the network logic

## Loss function 
Implement the SAL loss function

In [ ]:
 def evaluate_loss_sal(net, p, sigma, device, losses,batch_size=5000):
    ## Sample batch_size points, and then sample a random point around each point

    #sample points  around each of the samples

    

    # evaluate distances and compute the loss
    # compute and store the losses
    loss = 
    losses.append(loss.item())

    return loss

In [ ]:
def training_sal(point_cloud, loss_function, sigma=0.02):
    geomnet = SDFNet(3, gamma=0.5, sal_init=True)
    geomnet.to(device)

    pc_norm = get_normalized_pointcloud(point_cloud)
    points_torch = torch.from_numpy(pc_norm).float().to(device)

    lpc = []

    optim = torch.optim.Adam(params = geomnet.parameters(), lr=1e-4)

    nepochs=5000
    pbar = tqdm(total=nepochs,
                desc="Training")

    for epoch in range(nepochs):
        loss = loss_function(geomnet, points_torch, sigma, device, lpc, batch_size=5000)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if epoch % 100 == 0:
        #     print(f"Epoch {epoch}/{nepochs} - loss : {loss.item()}")
            pbar.set_postfix({'loss': loss.item()})
        pbar.update(1)


    return lpc, geomnet

In [ ]:
# If you get an error: did you modify SDFNet according to the instructions?
loss_sal, net_sal = training_sal(pc, evaluate_loss_sal, sigma=0.02)

In [ ]:
# Check that the network learned something
plt.figure(figsize=(6,4))
plt.yscale('log')
plt.plot(loss_sal, label = 'Point cloud loss ({:.2f})'.format(loss_[-1]))
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
u = compute_deepsdf(net_sal, 40)
vertices, triangles = mcubes.marching_cubes(u,0)
mesh = TriMesh(vertices, triangles)
plu.plot(mesh)

The produced signed distances using the proposed are too smooth and can't overfit a single shape. Therefore, the authors propose to learn the $L_0$ unsigned distance, by minimizing:

$$
\mathcal{L}(\theta) = \sum_i \mathbb{E}_{x \sim \mathcal{N}(x_i, \sigma^2)}[||u_{\theta}(x)| - 1|] + \mathbb{E}_{x \in \mathcal{X}}[|u_{\theta}(x)|],
$$

i.e. we want the distance to be $1$ outside of the surface, and $0$ on the surface.

Write the function evaluate_loss_sal_l0 below accordingly, and launch a new training to see the effects on the results.

In [ ]:
def evaluate_loss_sal_l0(net, p, sigma, device, losses,batch_size=5000):
    ## Do the sampling and evaluations

    # compute and store the losses
    loss =  

    losses.append(loss.item())

    return loss

In [ ]:
loss_sal_0, net_sal_0 = training_sal(pc, evaluate_loss_sal_l0, sigma=0.02)

In [ ]:
# Check that the network learned something
plt.figure(figsize=(6,4))
plt.yscale('log')
plt.plot(loss_sal_0, label = 'Point cloud loss ({:.2f})'.format(loss_sal_0[-1]))
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
u = compute_deepsdf(net_sal_0, 40)
vertices, triangles = mcubes.marching_cubes(u,0)
mesh = TriMesh(vertices, triangles)
plu.plot(mesh)

You can play with the sigma parameter to improve the results (see the paper to choose it wisely).